# SRU-Abfrage und Visualisierung von Daten aus dem VD17

Das Verzeichnis der im deutschen Sprachraum erschienenen Drucke des 17. Jahrhunderts (VD17) ist der zentrale Katalog für historische Drucke des 17. Jahrhunderts in deutschen Einrichtungen. Die Datensätze können über die SRU-Schnittstelle abgefragt und ausgewertet werden. 

Dieses Tutorial stellt die Schnittstellenabfrage des VD17 via SRU vor und gibt einen Einblick in die Visualisierungsmöglichkeiten. Grundsätzlich ist dieses Notebook auf für den VD18-Katalog nutzbar.

Codebasis ist die Script-Version, siehe https://github.com/F-Quaasdorf/VD17sru

# *Inhaltsverzeichnis*

**I. SRU-Abfrage und Datengrundlage**
1. Import der benötigten Module
2. Schnittstellenabfrage
3. Feldabfrage
4. Umwandlung in DataFrame
5. Ausführung und Datengrundlage

**II. Visualisierung der Daten**
1. Hilfsfunktion zur Konversion der Jahresangaben
2. Visualisierung zu den haltenden Einrichtungen
3. Visualisierung zu den Erscheinungsjahren
4. Visualisierung zu den Sprachen
5. Visualisierung zu den Sprachen nach Erschinungsjahren

## I. SRU-Abfrage und Datengrundlage

### 1. Import der benötigen Module

Für die SRU-Abfrage sowie die nachfolgende Datenbereinigung und -visualisierung werden mehrere Bibliotheken benötigt. Sie sind alle standardmäßig in Anaconda enthalten (Stand Sommer 2024). Für die Visualisierung wird Plotly genutzt.

In [1]:
import requests
import unicodedata
import ast
import re
import pandas as pd
import plotly.express as px
import plotly.io as pio
from lxml import etree

### 2. Schnittstellenabfrage

Die SRU-Schnittstelle kann direkt über die Browserzeile angesprochen werden. Die folgende Funktion basiert auf der SRU-Abfrage des [DNBLab](https://github.com/deutsche-nationalbibliothek/dnblab) und kreiert den dafür benötigten Suchstring:

In [2]:
def vd17_sru(query):    
    base_url = "http://sru.k10plus.de/vd17" # Für VD18: http://sru.k10plus.de/vd18
    parameters = {
        "recordSchema": "marcxml",
        "operation": "searchRetrieve",
        "version": "2.0",
        "maximumRecords": "100",
        "query": query
    }
    
    session = requests.Session()
    records = []
    start_record = 1
    first_request = True
    
    while True:
        parameters["startRecord"] = start_record
        response = session.get(base_url, params=parameters)
        if first_request:
            print(response.url)
            first_request = False
            
        if response.status_code != 200:
            print(f"Error fetching data: {response.status_code}")
            break
        
        xml_root = etree.fromstring(response.content)
        
        new_records = xml_root.findall('.//zs:record', namespaces={"zs": "http://docs.oasis-open.org/ns/search-ws/sruResponse"})
        
        for record in new_records:
            record_data = record.find('zs:recordData', namespaces={"zs": "http://docs.oasis-open.org/ns/search-ws/sruResponse"})
            if record_data is not None:
                marc_record = record_data.find('record', namespaces={"": "http://www.loc.gov/MARC21/slim"})
                if marc_record is not None:
                    records.append(etree.tostring(marc_record, encoding='unicode'))
        
        if len(new_records) < 100:
            break
        
        start_record += 100
    
    return records

Die Daten werden über `"recordSchema": "marcxml"` im Format MARC21-XML abgefragt. 
Es können maximal 100 Treffer auf einmal abgefragt werden. Die `while True`-Schleife sorgt dafür, dass alle Datensätze erfasst werden, indem der Paramter `startRecord` so lange wie nötig um 100 erweitert wird.

Anders als die MARC21-Datensätze aus anderen Bibliothekskatalogen (DNB, K10plus) besitzt die Struktur der VD17-MARC21 den zusätzlichen Namespace `zs`, der über die Schleife `for record in new_records:` aufgelöst wird.

### 3. Feldabfrage

Die folgende Funktion ruft die Daten aus den benötigten Feldern auf. In diesem Beispiel werden die Datan für die IDN, die VD-Nummer, der Verfasser, Titel, Erscheinungsort, Erscheinungsjahr, Sprache und die besitzende Einrichtung erfasst:

In [3]:
def parse_record(record):    
    namespaces = {
        "marc": "http://www.loc.gov/MARC21/slim"
    }
    xml = etree.fromstring(unicodedata.normalize("NFC", record))
    
    def get_single_text(xpath_expr):
        try:
            return xml.xpath(xpath_expr, namespaces=namespaces)[0].text
        except IndexError:
            return "N.N."
    
    def get_multiple_texts(xpath_expr):
        return [elem.text for elem in xml.xpath(xpath_expr, namespaces=namespaces)] or ["N.N."]
    
    meta_dict = {
        "VD-Nummer": get_single_text("//marc:datafield[@tag='024']/marc:subfield[@code='a']"),
        "Verfasser": get_single_text("//marc:datafield[@tag='100']/marc:subfield[@code='a']"),
        "Titel": get_single_text("//marc:datafield[@tag='245']/marc:subfield[@code='a']"),
        "Erscheinungsort": get_multiple_texts("//marc:datafield[@tag='264']/marc:subfield[@code='a']"),
        "Erscheinungsjahr": get_single_text("//marc:datafield[@tag='264']/marc:subfield[@code='c']"),
        "Sprache": get_multiple_texts("//marc:datafield[@tag='041']/marc:subfield[@code='a']"),
        "Einrichtung": get_multiple_texts("//marc:datafield[@tag='924']/marc:subfield[@code='b']")
    }
    
    return meta_dict

MARC21 besteht aus Controlfields, Datafields, die in Subfields unterteilt sein können. Die Verfasserangabe befindet sich beispielsweise im `datafield[@tag='100']` und dort im `subfield[@code='a']`. Welche Felder abgefragt werden sollen, kann beliebig angepasst werden. Es empfiehlt sich ein Blick in die XML-Datei.

Manche Felder sind wiederholbar, etwa wenn ein Werk in mehreren Orten veröffentlich worden ist. Um alle Orte zu erhalten, sind innerhalb der Funktion mit `get_single_text`und `get_multiple_texts` zwei Funktionen definiert, die entweder nur das erste (und im besten Falle einzige) Element aus dem Datensatz ziehen oder sämtliche Datensätze als Liste. 
Vorsicht, die Listen sind nicht verschachtelt! Sollte eine Liste `['Jena', '[Leipzig]', 'Dresden']` aussehen, dann bedeutet das, dass Leipzig als Veröffentlichungsort ermittelt worden ist und deswegen in Klammern steht. Diese Klammern sind Teil des Strings.

### 4. Umwandlung in DataFrame

In diesem Tutorial wird mit Pandas DataFrame gearbeitet. Geplant ist, diese in Zukunft durch Polars DataFrames zu ersetzen, die effizienter und schneller sind. Das wird aller Voraussicht nach zuerst in der [Script-Version](https://github.com/F-Quaasdorf/VD17sru) umgesetzt werden.

In [4]:
def to_df(records):    
    return pd.DataFrame(records)

Diese Funktion macht nichts anderes als die Ergebnisse aus den beiden bisherigen Funktionen in ein DataFrame umzuwandeln.

### 5. Ausführung und Datengrundlage

Die Abfrage kann durchgeführt werden. Für die Abfrage aus dem VD17 werden die Suchbegriffe nach PICA genommen:
- `pica.tit` für den Titel
- `pica.jah` für Jahresangaben
- `pica.per` für Personen
- `pica.vlo` für Ortsangaben

Im folgenden Beispiel werden Werke mit dem Titel "de statu imperii" gesucht, das DataFrame angezeigt und als CSV gespeichert.

In [5]:
records = vd17_sru("pica.tit=reich")

http://sru.k10plus.de/vd17?recordSchema=marcxml&operation=searchRetrieve&version=2.0&maximumRecords=100&query=pica.tit%3Dreich&startRecord=1


In [6]:
parsed_records = [parse_record(record) for record in records]
df = to_df(parsed_records)

In [7]:
pd.set_option('display.max_columns', None) # Wenn der Befehl 'print(df)' verwendet wird, werden so alle Spalten angezeigt.
df

,VD-Nummer,Verfasser,Titel,Erscheinungsort,Erscheinungsjahr,Sprache,Einrichtung
0,12:747849D,N.N.,Continuation Oder Fortsetzung der curieusen St...,[Cölln],1700,[ger],[N.N.]
1,3:743811F,N.N.,Kurtze auß denen heylsamen Reichs-Satzungen/ u...,[[S.l.]],[ca. 1700],[ger],[DE-3]
2,5002:735519F,"Lichtscheid, Ferdinand Helffreich",Die reine Absicht auf die Seeligkeit/ aus Vera...,[Zeitz],[ca. 1700],[ger],"[DE-547, DE-Ha32]"
3,12:733488E,N.N.,Monatlicher Staats-Spiegel,[Augspurg],1700,[ger],[N.N.]
4,12:733455N,N.N.,Monatlicher Staats-Spiegel,[Augspurg],1700,[ger],[N.N.]
...,...,...,...,...,...,...,...
2136,3:300733U,"Knaust, Heinrich",Feuwerzeugk Gerichtlicher Ordnunge Proceß unnd...,[Franckfort am Mayn],1601,[ger],"[DE-23, DE-3]"
2137,23:279150M,N.N.,Ottomannus Theologicus,[Eißleben],1601,[ger],"[DE-1a, DE-23, DE-23, DE-Ha32]"
2138,VD16 K 222,"Fickler, Johann Baptist",Klagschrifft Uber den Hochschädlichen Verlust ...,[München],1615 [i.e. 1595],[ger],"[DE-547, DE-3]"
2139,VD16 K 222,"Fickler, Johann Baptist",Klagschrifft Uber den Hochschädlichen Verlust ...,[München],1615 [i.e. 1595],[ger],[N.N.]


In [ ]:
df.to_csv("DataFrame.csv", encoding="utf-8")

## II. Visualisierung der Daten

### 1. Hilfsfunktion zur Konversion der Jahresangaben

Die Jahresangaben im VD17 sind gemäß der Angabe auf dem Werk eingetragen worden, sodass für eine Jahresabfrage neben vierstelligen Zahlen auch mit römischen Zahlen - hier auch irreguläre Angaben wie "IIII" statt "IV" - oder Versatzstücken wie "Im Jahre 1623" bzw. "Im Jahre MDCXXIII" umgegangen werden muss. 

In [ ]:
def convert_year(value):
    def convert_roman(roman):
        roman_values = {"I": 1, "V": 5, "X": 10, "L": 50, "C": 100, "D": 500, "M": 1000}
        
        total = 0
        prev_value = 0
        
        for char in reversed(roman):
            value = roman_values.get(char, 0)
            if value < prev_value:
                total -= value
            else:
                total += value
            prev_value = value
        
        return str(total)

    # Define a helper function to check if a string is a valid Roman numeral
    def is_roman_numeral(s):        
        # Matches valid and irregular Roman numerals, e.g. 'IIII' instead of 'IV'
        pattern = '^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,4})(IX|IV|V?I{0,4})$'
        
        return bool(re.match(pattern, s))
    
    original_value = value  # Keep the original value for debugging
    
    # Extract the first part if it contains additional information in brackets
    if "[" in value:
        match = re.search(r'\d{4}', value)
        if match:
            value = match.group(0)
        else:
            value = value.split("[")[0].strip()
    elif "-" in value:
        value = value.split("-")[0].strip()
    
    # Remove common non-Roman numeral prefixes
    value = re.sub(r'Im|Christi|De|Dato|Iahr|Den|Das', '', value, flags=re.IGNORECASE).strip()
    clean_value = ""
    
    match = re.search(r'\d{4}', value)
    if match:
        result = match.group(0)
    elif not value.isdigit():
        # Remove non-digit and non-Roman numeral characters
        clean_value = re.sub(r'[^IVXLCDM0-9]', '', value)
        if is_roman_numeral(clean_value):
            result = convert_roman(clean_value)
        else:
            result = value
    
    # Debugging output
    if result == '0':
        print(f"Debug: original_value='{original_value}', clean_value='{clean_value}', result='{result}'")
    
    return result

Die Funktion `convert_roman` wandelt sämtliche römischen Zahlen um. Über `is_roman_numeral` werden Jahres- und römische Zahlen extrahiert und normalisiert. Bei Zeitspannen wird stets die erste Zahl genommen. Die Funktion deckt fast alle möglichen Fälle ab. Ausnahmen werden im Original angezeigt und können über den Debug-output ermittelt werden.

### 2. Visualisierung zu den haltenden Einrichtungen

Diese Funktion sortiert die die Treffer nach Einrichtungen und zeigt, welche Institution wie viele Exemplare besitzt, die über die SRU-Abfrage gefunden wurden. Die Einrichtungen sind mit ihrem [Sigel](https://sigel.staatsbibliothek-berlin.de/startseite) angegeben. 

Die Einrichtungen liegen pro Werk als Liste vor, da jedes Werk in mehreren Institutionen vorhanden sein kann. Die Funktion löst sntprechend diese Listen nach den Elementen auf, gruppiert sie und gibt die Anzahl der Einrichtungen wieder aus.

In [ ]:
def location_graph(df):
    library_codes = df["Einrichtung"]
    
    library_list = []
    
    for element in library_codes:        
        if not isinstance(element, list):
            try:
                element = ast.literal_eval(element)
            except (ValueError, SyntaxError):
                continue
        for library in element:
            library_list.append(library)
    
    library_counts = pd.Series(library_list).value_counts()
    
    fig_loc = px.bar(library_counts, x=library_counts.index, y=library_counts.values,
                     labels={'x': 'Einrichtung', 'y': 'Anzahl'}, title='Anzahl der Werke nach Einrichtungen')
    
    return fig_loc

Die `for`-Schleife sorgt dafür, dass falls die Elemente der Spalte `df["Einrichtungen"]` nur als Strings und nicht als Liste vorliegen, sie trotzdem als solche behandelt werden, um die einzelnen Elemente zu erhalten. 

In [ ]:
fig_loc = location_graph(df)
fig_loc.show()

### 3. Visualisierung zu den Erscheinungsjahren

Hier werden die vorhandenen Treffer nach Erscheinungsjahren gruppiert und angezeigt.

In [ ]:
def publication_date_graph(df):
    dates = df["Erscheinungsjahr"].apply(convert_year)
    date_counts = dates.value_counts(dropna=False).sort_index()
    
    fig_dates = px.bar(date_counts, x=date_counts.index, y=date_counts.values,
                       labels={'x': 'Erscheinungsjahr', 'y': 'Anzahl'},
                       title='Anzahl der Werke nach Veröffentlichungsjahr')
    
    return fig_dates

In [ ]:
fig_dates = publication_date_graph(df)
fig_dates.show()

### 4. Visualisierung zu den Sprachen

Dieses Säulendiagramm gibt die Verteilung der genutzten Sprachen innerhalb der Treffermenge an. Dabei werden *nur* die Sprachen und ihre Häufigkeit beachtet. Ist bei einer Publikation sowohl `lat` für Latein als auch `ger` für Deutsch eingetragen, wird sie sowohl für Latein als auch für Deutsch gezählt. 

In [ ]:
def language_graph(df):
    language_codes = df["Sprache"]
    language_list = []
    
    for element in language_codes:        
        if not isinstance(element, list):
            try:
                element = ast.literal_eval(element)
            except (ValueError, SyntaxError):
                continue
        for language in element:
            language_list.append(language)
    
    language_counts = pd.Series(language_list).value_counts()
    
    fig_lang = px.bar(language_counts, x=language_counts.index, y=language_counts.values,
                      labels={'x': 'Sprache', 'y': 'Count'},
                      title='Anzahl der Werke nach Sprache')
        
    return fig_lang

In [ ]:
fig_lang = language_graph(df)
fig_lang.show()

### 5. Visualisierung zu den Sprachen nach Erschinungsjahren

Hier wird das Vorkommen der Sprachen aus der Treffermenge im Laufe der Zeit angegeben.

In [ ]:
def language_year_graph(df):
    df['Cleaned_Year'] = df['Erscheinungsjahr'].apply(convert_year)
        
    df_exploded = df.explode('Sprache')    
    language_year_counts = df_exploded.groupby(['Cleaned_Year', 'Sprache']).size().reset_index(name='Count')

    fig_lang_year = px.line(language_year_counts, x='Cleaned_Year', y='Count', color='Sprache',
                            labels={'Cleaned_Year': 'Erscheinungsjahr', 'Count': 'Anzahl', 'Sprache': 'Sprache'},
                            title='Anzahl der verwendeten Sprachen über die Zeit')
    
    return fig_lang_year

In [ ]:
fig_lang_year = language_year_graph(df)
fig_lang_year.show()